In [13]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [14]:
#load 20NG
# train_20newsgroups = fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),shuffle=True, random_state=42)
# test_20newsgroups =  fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),shuffle=True, random_state=42)
# vectorizer = TfidfVectorizer()
# train = vectorizer.fit_transform(train_20newsgroups.data)
# vocab = vectorizer.get_feature_names()
# vectorizer2=TfidfVectorizer(vocabulary=vocab)
# test = vectorizer2.fit_transform(test_20newsgroups.data)

twentyng_data = fetch_20newsgroups(subset='all',remove=('headers','footers','quotes'))
vectorizer = TfidfVectorizer(stop_words='english',max_df=0.95,min_df=2)
twentyng_data_vectors = vectorizer.fit_transform(twentyng_data.data)
twentyng_target = twentyng_data.target
tNGX_train, tNGX_test, tNGy_train, tNGy_test = train_test_split(twentyng_data_vectors, twentyng_target, test_size=0.33, random_state=42)

In [15]:
# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 200
display_step = 100

# Network Parameters
n_hidden_1 = 500 # 1st layer number of neurons
n_hidden_2 = 500 # 2nd layer number of neurons
#num_input = 784 # 
num_classes = 20 

In [16]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'data': tNGX_train.todense()}, y=tNGy_train,
    batch_size=batch_size, num_epochs=None, shuffle=True)

In [17]:
# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['data']
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer

In [18]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    logits = neural_net(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [19]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmph7tvmnpe', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11de1da20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
# Train the Model
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmph7tvmnpe/model.ckpt.
INFO:tensorflow:loss = 2.995963424504656, step = 1
INFO:tensorflow:global_step/sec: 1.24181
INFO:tensorflow:loss = 2.9758638196505562, step = 101 (80.586 sec)


KeyboardInterrupt: 

In [12]:
# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'data': tNGX_test.todense()}, y=tNGy_test,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-22-21:45:30
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpu4nfy55y/model.ckpt-699
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-22-21:45:46
INFO:tensorflow:Saving dict for global step 699: accuracy = 0.45610932, global_step = 699, loss = 2.8193946


{'accuracy': 0.45610932, 'global_step': 699, 'loss': 2.8193946}